In [1]:
import cv2
import tensorflow as tf
import os 

class_names = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4',
               5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
               10: 'a', 11: 'b', 12: 'c', 13: 'd', 14: 'e',
               15: 'f', 16: 'g', 17: 'h', 18: 'i', 19: 'k',
               20: 'l', 21: 'm', 22: 'n', 23: 'o', 24: 'p',
               25: 'q', 26: 'r', 27: 's', 28: 't', 29: 'u',
               30: 'v', 31: 'w', 32: 'x', 33: 'y'}

In [2]:
# code to stack windows horizontally or vertically 
# taken from https://youtu.be/WQeoO7MI0Bs also can refer to the link in the video description

def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver


In [ ]:
# to be used to reset the GPU if required

from numba import cuda 
device = cuda.get_current_device()
device.reset()


In [ ]:



# gap to prevent confusion




In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers
from tensorflow.keras import Model

with tf.device('GPU:0'):
    # Using VGG19 model with Imagenet weights to extract features from the images for faster and better results.

    base_model = VGG19(include_top=False, weights='imagenet', input_shape=(136,260,3))

    for layer in base_model.layers:    # freezing the base_model layers
        layer.trainable= False

    # base_model.summary()

    last_output = base_model.output

    x = layers.Flatten()(last_output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)                
    x = layers.Dense(34, activation='softmax')(x)    # number of neurons to be changed depending upon number of output classes       

    model = Model( base_model.input, x) 

    model.compile(optimizer ='adam', 
                  loss = 'categorical_crossentropy', 
                  metrics = ['accuracy'])

    model.summary()

    model.load_weights('C:/Users/sinha/Desktop/Open cv/total_dataset_89/model_total_dataset_89.h5')

In [4]:
from efficientnet import tfkeras as efn
from tensorflow.keras import layers
from tensorflow.keras import Model


base_model = efn.EfficientNetB2(input_shape = (240, 460, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:    # freezing the base_model layers
    layer.trainable= False
    
# base_model.summary()

last_output = base_model.output

x = layers.Flatten()(last_output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)                
x = layers.Dense(34, activation='softmax')(x)    # number of neurons to be changed depending upon number of output classes       

model = Model( base_model.input, x) 

model.compile(optimizer ='adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

model.summary()

model.load_weights('efficientnetb2.h5')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 240, 460, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 120, 230, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 120, 230, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 120, 230, 32) 0           stem_bn[0][0]                    
______________________________________________________________________________________________

In [9]:
import numpy as np

cap = cv2.VideoCapture(0)  # 0 for laptop webcam and 1 for external webcam

while True:
    suc, img = cap.read()

#     width = 260
#     height = 136
#     dim = (width, height)

    # resize image
#     resized = cv2.resize(img[0:240,20:260], dim, interpolation = cv2.INTER_AREA)

#     cv2.rectangle(img,(20,0),(260,240),(0,255,0),3)

    cv2.rectangle(img,(20,0),(480,240),(0,255,0),3)
    
    img_array = tf.keras.preprocessing.image.img_to_array(img[0:240,20:480])
    img_array = tf.expand_dims(img_array, 0)   # Create a batch
    
    with tf.device('/CPU:0'):
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        print(score)

    cv2.putText(img, class_names[np.argmax(score)], (50,50), cv2.FONT_HERSHEY_SIMPLEX , 1, (0,0,255), 2, cv2.LINE_AA)

    imgStack = stackImages(0.6,[img,img[0:240,20:480]])
    cv2.imshow("Stacked Images", imgStack)   # displaying the stacked images


    key = cv2.waitKey(1)
    if key == ord('z'):  # if "z" is pressed the close the capture proccess
        flag=False
        cap.release()


cv2.destroyAllWindows()  # destroy/close the windows opened by the program

tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0

tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0

tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0

tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0

tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0

tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0

tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686], shape=(34,), dtype=float32)
tf.Tensor(
[0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686
 0.02799686 0.02799686 0.02799686 0.02799686 0.02799686 0.07610334
 0.02799686 0

TypeError: 'NoneType' object is not subscriptable